In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import cv2
from glob import glob
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow.keras.preprocessing import image
from matplotlib.image import imread
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import BatchNormalization
from sklearn.metrics import classification_report,confusion_matrix
from tensorflow.keras.models import Sequential, Model
from keras.regularizers import l2
from tensorflow.keras.layers import Activation, Dropout, Dense, Flatten, Conv2D, BatchNormalization, MaxPooling2D, GlobalAveragePooling2D,Input,concatenate, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Main Folder Path
folder_path = "/kaggle/input/kermany2018/OCT2017 "

# Sub Folder Paths
train_dir = f"{folder_path}/train"
val_dir = f"{folder_path}/val"
test_dir = f"{folder_path}/test"

In [ ]:
os.listdir(folder_path)

In [ ]:
print(f"Train Directory: {os.listdir(train_dir)}")
print(f"Validation Directory: {os.listdir(test_dir)}")
print(f"Test Directory: {os.listdir(val_dir)}")

In [ ]:
# === Column 2: Data generators (IRv2) + class weights ===
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np, os

IMG_SIZE   = (299, 299)   # Inception-ResNet-v2 native
BATCH_SIZE = 32

train_aug = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.05,
    height_shift_range=0.05,
    zoom_range=0.10,
    horizontal_flip=True,
)
eval_aug  = ImageDataGenerator()

train_gen = train_aug.flow_from_directory(
    train_dir, target_size=IMG_SIZE, color_mode="rgb",
    batch_size=BATCH_SIZE, class_mode="categorical", shuffle=True
)
val_gen = eval_aug.flow_from_directory(
    val_dir, target_size=IMG_SIZE, color_mode="rgb",
    batch_size=BATCH_SIZE, class_mode="categorical", shuffle=False
)
test_gen = eval_aug.flow_from_directory(
    test_dir, target_size=IMG_SIZE, color_mode="rgb",
    batch_size=BATCH_SIZE, class_mode="categorical", shuffle=False
)

# Compute class weights: total / (n_classes * count_c)
counts = np.zeros(train_gen.num_classes, dtype=np.int64)
for cls, idx in train_gen.class_indices.items():
    counts[idx] = len(os.listdir(os.path.join(train_dir, cls)))
total = int(counts.sum()); ncls = len(counts)
class_weights = {i: float(total/(ncls*counts[i])) for i in range(ncls)}

print("Counts:", counts.tolist())
print("Class weights:", class_weights)


In [ ]:
# === Column 3: Inception-ResNet-v2 model + Stage-1 (frozen) ===
import tensorflow as tf
from tensorflow.keras import layers, Model, optimizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, CSVLogger

num_classes = train_gen.num_classes

# Input + in-model rescale to [-1, 1] (keeps generators vanilla)
inp = layers.Input(shape=(*IMG_SIZE, 3), name="image")
x   = layers.Rescaling(1./127.5, offset=-1)(inp)

# Robust weights load (falls back to random init if download blocked)
try:
    base = tf.keras.applications.InceptionResNetV2(
        include_top=False, weights="imagenet", input_tensor=x
    )
    print("IRv2 ImageNet weights loaded.")
except Exception as e:
    print("IRv2 weights download issue -> random init used.", e)
    base = tf.keras.applications.InceptionResNetV2(
        include_top=False, weights=None, input_tensor=x
    )

y   = layers.GlobalAveragePooling2D()(base.output)
y   = layers.Dropout(0.3)(y)
out = layers.Dense(num_classes, activation="softmax")(y)
model = Model(inp, out, name="IRv2_OCT")
model.summary()

# Stage-1: freeze backbone
base.trainable = False
loss = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.10)

model.compile(optimizer=optimizers.Adam(1e-3),
              loss=loss, metrics=["accuracy"])

ckpt_s1 = "IRv2_stage1_best.keras"
cb_s1 = [
    EarlyStopping(monitor="val_loss", patience=4, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor="val_loss", factor=0.3, patience=2, verbose=1),
    ModelCheckpoint(ckpt_s1, monitor="val_accuracy", save_best_only=True, verbose=1),
    CSVLogger("IRv2_stage1_log.csv", append=False),
]

EPOCHS_S1 = 8
history_s1 = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS_S1,
    class_weight=class_weights,
    callbacks=cb_s1,
    verbose=1,
)
